compare calibration result and simulation result in the instant loop detector

In [71]:
import os
print("Current directory:", os.getcwd())
HORNSGATAN_HOME = os.environ["HORNSGATAN_HOME"]
os.chdir(HORNSGATAN_HOME)
print("Current directory:", os.getcwd())

#number = len(data)
import pandas as pd
date = '2020-01-01'
detector = 'e2w_out'
number = 100
path  = "data/sim_data/"
if number<1:
    postfix = f"{detector}_{date}"
else:
    postfix = f"{detector}_{date}_{number}"

instantInductionLoop_filename_xml = f"{path}instantInductionLoop_{postfix}.xml"


Current directory: /home/kaveh/projects/Hornsgatan
Current directory: /home/kaveh/projects/Hornsgatan


In [72]:
import xml.etree.ElementTree as ET

# Parse the XML content of output_detectors.xml
tree_output = ET.parse(instantInductionLoop_filename_xml)
root_output = tree_output.getroot()

# Extract data from the XML
instant_out_data = []
for instant_out in root_output.findall('instantOut'):
    instant_out_data.append({
        'id': instant_out.get('id'),
        'time': float(instant_out.get('time')),
        'state': instant_out.get('state'),
        'vehID': instant_out.get('vehID'),
        'speed': float(instant_out.get('speed')),  # Convert speed to km/h
        'length': instant_out.get('length'),
        'type': instant_out.get('type'),
        'gap': instant_out.get('gap')
    })

# Convert to a DataFrame
df_instant_out = pd.DataFrame(instant_out_data)
df_instant_out = df_instant_out[df_instant_out["state"]=='enter']
# Save to CSV
#output_csv_file_instant_out = 'output_instant_out.csv'
#df_instant_out.to_csv(output_csv_file_instant_out, index=False)
#print(f"Data from 'output_detectors.xml' successfully converted to CSV and saved as '{output_csv_file_instant_out}'.")


In [73]:
df_instant_out

,id,time,state,vehID,speed,length,type,gap
0,e2w_out,1.577837e+09,enter,0_e2w_out,8.16,5.00,DEFAULT_VEHTYPE,None
3,e2w_out,1.577837e+09,enter,1_e2w_out,11.26,5.00,DEFAULT_VEHTYPE,30.40
6,e2w_out,1.577837e+09,enter,2_e2w_out,4.17,5.00,DEFAULT_VEHTYPE,36.31
9,e2w_out,1.577837e+09,enter,3_e2w_out,9.10,5.00,DEFAULT_VEHTYPE,34.02
12,e2w_out,1.577837e+09,enter,4_e2w_out,10.05,5.00,DEFAULT_VEHTYPE,7.01
...,...,...,...,...,...,...,...,...
243,e2w_out,1.577843e+09,enter,95_e2w_out,7.38,5.00,DEFAULT_VEHTYPE,27.77
246,e2w_out,1.577843e+09,enter,96_e2w_out,7.38,5.00,DEFAULT_VEHTYPE,1.34
248,e2w_out,1.577843e+09,enter,97_e2w_out,10.86,5.00,DEFAULT_VEHTYPE,20.07
251,e2w_out,1.577843e+09,enter,98_e2w_out,10.74,5.00,DEFAULT_VEHTYPE,1.36


In [74]:
data = pd.read_csv(f'data/calibration_data/calibrated_data_{postfix}.csv')
data.rename(columns={"veh_id": "vehID"}, inplace=True)
data.head()

,vehID,time_detector_sim,speed_detector_sim,speed_factor,time_detector_real,depart,departSpeed,speed_detector_real,delta_time,delta_speed
0,0_e2w_out,1.577837e+09,8.157955,0.979346,1577836850,1577836821,max,7.777778,-0.04,0.380177
1,1_e2w_out,1.577837e+09,11.264705,1.352306,1577836881,1577836860,max,11.111111,-0.03,0.153594
2,2_e2w_out,1.577837e+09,4.165000,0.500000,1577836917,1577836860,max,3.611111,0.72,0.553889
3,3_e2w_out,1.577837e+09,9.104840,1.093018,1577836953,1577836927,max,9.166667,-0.05,-0.061827
4,4_e2w_out,1.577837e+09,10.050596,1.206554,1577836960,1577836937,max,9.722222,0.51,0.328374


In [75]:
print(len(data), len(df_instant_out))

100 100


compare time error and speed error in loop detector between simulation result and calibration result

In [76]:
compare_df = pd.merge(df_instant_out[["vehID","time","speed"]],
            data[["vehID","time_detector_sim","speed_detector_sim"
                  ,"time_detector_real","speed_detector_real"]],on="vehID")
compare_df.head()




,vehID,time,speed,time_detector_sim,speed_detector_sim,time_detector_real,speed_detector_real
0,0_e2w_out,1.577837e+09,8.16,1.577837e+09,8.157955,1577836850,7.777778
1,1_e2w_out,1.577837e+09,11.26,1.577837e+09,11.264705,1577836881,11.111111
2,2_e2w_out,1.577837e+09,4.17,1.577837e+09,4.165000,1577836917,3.611111
3,3_e2w_out,1.577837e+09,9.10,1.577837e+09,9.104840,1577836953,9.166667
4,4_e2w_out,1.577837e+09,10.05,1.577837e+09,10.050596,1577836960,9.722222


In [77]:
compare_df = compare_df.head(100)

In [78]:
time_calib_error = compare_df["time_calib_error"]=(compare_df["time_detector_sim"]-compare_df["time_detector_real"])
abs(time_calib_error).mean()

np.float64(0.6228999805450439)

In [79]:
time_sim_error = compare_df["time_sim_error"] = compare_df["time"]-compare_df["time_detector_real"]
abs(time_sim_error).mean()

np.float64(0.47159998178482054)

In [80]:
speed_calib_error= compare_df["speed_calib_error"] =compare_df["speed_detector_sim"]-compare_df["speed_detector_real"]
abs(speed_calib_error).mean()

np.float64(0.5038388430868519)

In [81]:
speed_sim_error=compare_df["speed_sim_error"] =compare_df["speed"]-compare_df["speed_detector_real"]
abs(speed_sim_error).mean()

np.float64(0.5361111111111111)

In [12]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [82]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-4,  # Bin start
        end=11,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['time_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['time_calib_error']).mean(),2)

fig.add_trace(go.Histogram(x=compare_df['time_sim_error'], name=f"time_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['time_calib_error'], name=f"time_calib_error  --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='time error (s)',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_time_error_{postfix}.html")

In [13]:
print(compare_df['time_sim_error'].min(), compare_df['time_sim_error'].max())
print(compare_df['time_calib_error'].min(), compare_df['time_calib_error'].max())

-1.7400000095367432 2.9100000858306885
-1.7400000095367432 5.950000047683716


In [14]:
print(compare_df['speed_sim_error'].min(), compare_df['speed_sim_error'].max())
print(compare_df['speed_calib_error'].min(), compare_df['speed_calib_error'].max())

-1.9511111111111106 5.555555555555555
-0.8389999999999969 5.552555555555555


In [44]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [83]:
import plotly.graph_objects as go

fig = go.Figure()

xbins=dict(
        start=-7,  # Bin start
        end=7,    # Bin end
        size=1    # Bin size (width of each bin)
    )
sim_MAE = round(abs(compare_df['speed_sim_error']).mean(),2)
calib_MAE = round(abs(compare_df['speed_calib_error']).mean(),2)
fig.add_trace(go.Histogram(x=compare_df['speed_sim_error'], name=f"speed_sim_error --> MAE = {sim_MAE}", opacity=0.75, xbins=xbins))
fig.add_trace(go.Histogram(x=compare_df['speed_calib_error'], name=f"speed_calib_error --> MAE = {calib_MAE}", opacity=0.75, xbins=xbins))

fig.update_layout(
    barmode='group',  # Use 'overlay' if you want stacked look
    xaxis_title='Value',
    yaxis_title='Count',
    title=f"error  =  simulate  -  real      ------------         {postfix}"
)

fig.show()
fig.write_html(f"diagram/hist_speed_error_{postfix}.html")

In [16]:
import os
os.environ["HORNSGATAN_HOME"]


'/home/kaveh/projects/Hornsgatan/'